## CSE5ML_Lab_9A: Some popular network network structures for image classification
VGGNet, ResNet, Inception, and Xception are four types popular neural networks that are proven to be effective in image classification tasks. In keras, there are five popular model structures, namely VGG16, VGG19, ResNet50, Inception V3 and Xception; you can train these models from scratch with newly initialized weights, or you can load pretrained model, based on the ImageNet dataset (another popular benchmark dataset in image classification, which is even larger than the CIFAR10 dataset, with a size of 224*224 for each image). Sometimes we find the pretrained model very helpful when the input data is similar and we do not want to use a lot of time to retrain the model from scratch.

If you are interested in more information about these models and implementation with Keras, you can check this link: https://www.pyimagesearch.com/2017/03/20/imagenet-vggnet-resnet-inception-xception-keras/

Here we use VGG16 as an example. The same steps can be applied in other models.

The below attached is the model achitecture for the original VGG16: 

    # Block 1
    x = layers.Conv2D(64, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block1_conv1')(img_input)
    x = layers.Conv2D(64, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block1_conv2')(x)
    x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = layers.Conv2D(128, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block2_conv1')(x)
    x = layers.Conv2D(128, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block2_conv2')(x)
    x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = layers.Conv2D(256, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block3_conv1')(x)
    x = layers.Conv2D(256, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block3_conv2')(x)
    x = layers.Conv2D(256, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block3_conv3')(x)
    x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = layers.Conv2D(512, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block4_conv1')(x)
    x = layers.Conv2D(512, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block4_conv2')(x)
    x = layers.Conv2D(512, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block4_conv3')(x)
    x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    x = layers.Conv2D(512, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block5_conv1')(x)
    x = layers.Conv2D(512, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block5_conv2')(x)
    x = layers.Conv2D(512, (3, 3),
                      activation='relu',
                      padding='same',
                      name='block5_conv3')(x)
    x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)


    # Classification block
    x = layers.Flatten(name='flatten')(x)
    x = layers.Dense(4096, activation='relu', name='fc1')(x)
    x = layers.Dense(4096, activation='relu', name='fc2')(x)
    x = layers.Dense(classes, activation='softmax', name='predictions')(x)

### Load dataset and Preprocess data

In [1]:
# packages
import numpy as np
import tensorflow as tf
from keras.datasets import cifar10
from keras.utils import np_utils

# load data
(Inputs, Labels), (Test_Data, Test_Label) = cifar10.load_data() # notice the first line of importing packages

# normalize inputs from 0-255 to 0.0-1.0
# Neural networks process inputs using small weight values, and inputs with large integer values can disrupt or slow down the learning process. As such it is good practice to normalize the pixel values so that each pixel value has a value between 0 and 1.
Inputs = Inputs.astype('float32')
Test_Data = Test_Data.astype('float32')
Inputs = Inputs / 255.0
Test_Data = Test_Data / 255.0

# Encode the outputs with one hot coding
Labels = np_utils.to_categorical(Labels) #Converts a class vector (integers) to binary class matrix.
Test_Label = np_utils.to_categorical(Test_Label)
num_classes = Test_Label.shape[1]

Using TensorFlow backend.


### Load Model
Here I give two ways to load the model structure and train the model on CIFAR10 dataset. First, if you do not care about how much time or resources to use on trainig, you can train the model from scratch with newly intialized weights. Another way is to train the model based some pretrained weights on similar datasets (because it is found that the first few layers trained for different dataset basically do the similar things, and this's the reason we can consider adopt the weights trained from other dataset, and further fine-tune the weights on our dataset). This can accelate training with fewer epochs/iterations. 

Note: when apply the below codes, you may find your computer resources can not really support you running them, and here I just show you how you can load a pretrained model weights which has been trained on CIFAR10 already, and you can see how this complex neural network structure can improve classification performance.

In [ ]:
# train model from scratch

# load vgg model
from keras.applications.vgg16 import VGG16

# load the model
model = VGG16(weights=None, include_top=False, input_shape=(32, 32, 3))
model.summary()

In [ ]:
# load pretrained model and finetune
# Note that we drop the 3 fully-connected layers at the top of the network which mainly act like classifiers to classify the extracted features from the convolutional layers, because we have a new dataset and we want to train a new classifier.

# load vgg model
from keras.applications.vgg16 import VGG16

# load the model
model = VGG16(weights="imagenet", include_top=False, input_shape=(32, 32, 3))
model.summary()

In [2]:
# load pretrained model which has been finetuned on CIFAR10, not that this model has only used the first 3 blocks in the VGG16 model
# load vgg model
from keras.applications.vgg16 import VGG16
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.engine import Model

# load the model
model = VGG16(weights=None, include_top=False, input_shape=(32, 32, 3))
# Extract the last layer from third block of vgg16 model
last = model.get_layer('block3_pool').output
# Add classification layers on top of it
x = Flatten()(last)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
pred = Dense(10, activation='softmax')(x)
model = Model(model.input, pred)

# load pretrained weigths
model.load_weights('cifar10-vgg16_model.h5')

# summarize the model
model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
___________________________________________

### Compile the model

In [3]:
# Compile model
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

### Train the model
You do not need to run this if you just want to evaluate a pretrained model and you not want to train or fine-tune a model, 

In [ ]:
tf.set_random_seed(1)
np.random.seed(1)

epochs = 10
# Fit the model
model.fit(Inputs, Labels, validation_data=(Test_Data, Test_Label), epochs=epochs, batch_size=32, verbose=1)

you can save the model weight to use it next time

In [4]:
model.save_weights('cifar10_vgg16_new_model.h5')

### Evaluate the model with testing dataset

In [4]:
# Final evaluation of the model
scores = model.evaluate(Test_Data, Test_Label, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))


Accuracy: 86.40%
